In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from glob import glob
import os

In [10]:
device = torch.device('cuda')
dtype = torch.float32

In [11]:
train_labels = np.load('/home/max/saved_data/train_repre/tr_labels.npy')
test_labels = np.load('/home/max/saved_data/test_repre/te_labels.npy')

In [12]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, train=True):
        super().__init__()
        self.train = train
        if train == True:
            self.all_repre = sorted(glob(os.path.join(root_dir, 'train_*.npy')))
        if train == False:
            self.all_repre = sorted(glob(os.path.join(root_dir, 'test_*.npy')))
            
    def __len__(self):
        return len(self.all_repre)        
    
    def __getitem__(self, idx):
        repre = np.load(self.all_repre[idx]).squeeze()
        if self.train == True:
            return (repre, train_labels[idx])
        if self.train == False:
            return (repre, test_labels[idx])

In [13]:
train_dataset = Dataset(os.path.join('/home/max/saved_data/train_repre'), True)
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)
test_dataset = Dataset(os.path.join('/home/max/saved_data/test_repre'), False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [14]:
def train(model, optimizer, epochs=1):

    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x,y) in enumerate(train_loader):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % 20 == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                
        torch.save(model.state_dict(), os.path.join('/home/max/saved_data/classifier/model_epoch'+str(e)+'.pth'))

In [15]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
    
        self.conv1 = nn.Conv3d(64, 64, kernel_size = 3, padding = 1)
        self.conv2 = nn.Conv3d(64, 64, kernel_size = 3, padding = 1)
        self.conv3 = nn.Conv3d(64, 64, kernel_size = 3, padding = 1)
        self.conv4 = nn.Conv3d(64, 64, kernel_size = 3, padding = 1)
        self.fc1 = nn.Linear(64*2*2*2, 64)
        self.fc2 = nn.Linear(64, 13)
        self.maxpool = nn.MaxPool3d(2, stride=2, padding=0)
        
        
    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        out = self.maxpool(out)
        out = self.conv2(out)
        out = F.relu(out)
        out = self.maxpool(out)
        out = self.conv3(out)
        out = F.relu(out)
        out = self.maxpool(out)
        out = self.conv4(out)
        out = F.relu(out)
        out = self.maxpool(out)
        
        out = out.view(out.shape[0],-1)
        out = self.fc1(out)
        out = F.relu(out)
        scores = self.fc2(out)
        return scores

In [16]:
learning_rate = 1e-3
model = ConvNet()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


train(model, optimizer, epochs=30)

Iteration 0, loss = 2.5198
Iteration 20, loss = 2.6036
Iteration 40, loss = 2.6084
Iteration 60, loss = 2.1930
Iteration 80, loss = 2.5093
Iteration 100, loss = 2.4267
Iteration 120, loss = 1.6189
Iteration 140, loss = 2.1803
Iteration 160, loss = 1.5879
Iteration 180, loss = 2.4693
Iteration 200, loss = 1.7325
Iteration 0, loss = 0.6517
Iteration 20, loss = 2.7580
Iteration 40, loss = 0.2630
Iteration 60, loss = 1.1348
Iteration 80, loss = 0.8590
Iteration 100, loss = 0.2543
Iteration 120, loss = 0.4734
Iteration 140, loss = 1.3644
Iteration 160, loss = 0.3079
Iteration 180, loss = 0.7578
Iteration 200, loss = 0.5828
Iteration 0, loss = 0.5882
Iteration 20, loss = 0.4171
Iteration 40, loss = 0.3173
Iteration 60, loss = 0.8344
Iteration 80, loss = 0.6031
Iteration 100, loss = 0.5487
Iteration 120, loss = 0.3258
Iteration 140, loss = 0.4382
Iteration 160, loss = 0.7018
Iteration 180, loss = 1.5920
Iteration 200, loss = 1.3183
Iteration 0, loss = 0.8249
Iteration 20, loss = 0.3855
Iterat

Iteration 40, loss = 0.0077
Iteration 60, loss = 0.0019
Iteration 80, loss = 0.0000
Iteration 100, loss = 0.0151
Iteration 120, loss = 0.0000
Iteration 140, loss = 0.0125
Iteration 160, loss = 0.0000
Iteration 180, loss = 0.0030
Iteration 200, loss = 0.1791
Iteration 0, loss = 0.0009
Iteration 20, loss = 0.0000
Iteration 40, loss = 0.0170
Iteration 60, loss = 0.0033
Iteration 80, loss = 0.0000
Iteration 100, loss = 0.0001
Iteration 120, loss = 0.0139
Iteration 140, loss = 0.0000
Iteration 160, loss = 0.0095
Iteration 180, loss = 2.0894
Iteration 200, loss = 0.0022
Iteration 0, loss = 0.0035
Iteration 20, loss = 0.0038
Iteration 40, loss = 0.0004
Iteration 60, loss = 0.0004
Iteration 80, loss = 0.0000
Iteration 100, loss = 0.0000
Iteration 120, loss = 0.0000
Iteration 140, loss = 0.0000
Iteration 160, loss = 0.0003
Iteration 180, loss = 0.0000
Iteration 200, loss = 0.0000
Iteration 0, loss = 0.0000
Iteration 20, loss = 0.0000
Iteration 40, loss = 0.0000
Iteration 60, loss = 0.0000
Itera

In [17]:
def check_accuracy_part(loader, model): 
    num_correct = 0
    num_samples = 0
    model = model.to(device=device)
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        return acc

In [21]:
accs = list()
for i in range(0,30,2):
    model = ConvNet()
    model.load_state_dict(torch.load(os.path.join('/home/max/saved_data/classifier/model_epoch'+str(i)+'.pth')))
    acc = check_accuracy_part(test_loader, model)
    accs.append(acc)

accs = np.array(accs)

Got 160 / 260 correct (61.54)
Got 184 / 260 correct (70.77)
Got 201 / 260 correct (77.31)
Got 199 / 260 correct (76.54)
Got 195 / 260 correct (75.00)
Got 199 / 260 correct (76.54)
Got 203 / 260 correct (78.08)
Got 201 / 260 correct (77.31)
Got 200 / 260 correct (76.92)
Got 200 / 260 correct (76.92)
Got 205 / 260 correct (78.85)
Got 195 / 260 correct (75.00)
Got 203 / 260 correct (78.08)
Got 204 / 260 correct (78.46)
Got 207 / 260 correct (79.62)


In [ ]:
np.save(os.path.join(),accs)